Pretrained T5 based Experiment
Raw Dataset



## Total Object Models: 9


## Training

1.	Camping
2.	Customer_Order
3.	Ecommerce
4.	Onlinestore
5.	Decider
6.  Library OM
7.  CSOS
8.  Flagship

## Testing

9.	Bank





--------------------------------

## Total Training Data: 13236 (100%)
--------------------------------

### Training set P :  9265 (70% of Training Data)

### Training set NP : 3971 (30% of Training Data)

-----------------------------
## Total Testing Data: 32 (100% of Total Data)
----------------------------

### Testing set P : 8 (25% of Testing Data)

### Testing set NP : 24 (75%% of Testing Data)

In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
pip install accelerate -U

In [ ]:
#Import Libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch

In [ ]:
import pandas as pd

# Load the dataset from the first sheet of the Excel file
data = pd.read_csv("raw_8_om_training_set.csv")

In [ ]:
# Selecting the relevant columns
data = data[['OM_Regular', 'OM_Prediction']]

In [ ]:
# Splitting data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Load the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenizer.model_max_length = 512  # Set maximum sequence length directly in the tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

   def preprocess_data(data, tokenizer, max_seq_length=512):
    input_texts = list(data['OM_Regular'])
    target_texts = list(data['OM_Prediction'])

    # Tokenize input and target texts
    input_encodings = tokenizer(input_texts, truncation=True, padding='max_length', max_length=max_seq_length)
    target_encodings = tokenizer(target_texts, truncation=True, padding='max_length', max_length=max_seq_length)
        # Convert tokenized sequences to PyTorch tensors
    input_ids = torch.tensor(input_encodings['input_ids'])
    target_ids = torch.tensor(target_encodings['input_ids'])

    return input_ids, target_ids

train_inputs, train_targets = preprocess_data(train_data, tokenizer)
test_inputs, test_targets = preprocess_data(test_data, tokenizer)

In [ ]:
# Define a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'labels': self.targets[idx]
        }


In [ ]:
# Create instances of custom dataset
train_dataset = CustomDataset(train_inputs, train_targets)
eval_dataset = CustomDataset(test_inputs, test_targets)


In [ ]:

# Define training arguments with smaller batch size
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=8,  # Reduce batch size
    per_device_eval_batch_size=8,   # Reduce batch size for evaluation
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    save_steps=1000,
    evaluation_strategy="epoch",
)

In [ ]:
# Define a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.004300,0.001307
2,0.002500,0.001232
3,0.001600,0.001212
4,0.001500,0.001186
5,0.001400,0.001193
6,0.001300,0.001195
7,0.001300,0.001187
8,0.001300,0.001185


TrainOutput(global_step=10672, training_loss=0.17859919043327646, metrics={'train_runtime': 8062.9586, 'train_samples_per_second': 10.585, 'train_steps_per_second': 1.324, 'total_flos': 1.1550610704826368e+16, 'train_loss': 0.17859919043327646, 'epoch': 8.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate(eval_dataset)

In [ ]:
# # Calculate additional metrics
# labels = test_targets.flatten().tolist()
# preds = model.generate(test_inputs)
# preds = torch.tensor(preds).flatten().tolist()
# accuracy = accuracy_score(labels, preds)
# precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

# # Display evaluation results
# print("Evaluation Results:")
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

In [ ]:
# Test the model with new input data
def generate_predictions(input_text, model, tokenizer):
    # Move input text to the same device as the model
    device = next(model.parameters()).device

    # Encode input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate predictions
    with torch.no_grad():
        outputs = model.generate(input_ids.to('cuda:0'))  # Move input_ids to CPU before generation

    # Move predictions to CPU and decode
    predicted_text = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)

    return predicted_text

# Test the model with new input data
input_text = "module ecommerceopen Declarationone sig Customer extends Class{}{attrSet = customerIDid=customerIDisAbstract = Nono parent}one sig customerID extends Integer{}one sig Order extends Class{}{attrSet = orderIDid=orderIDisAbstract = Nono parent}one sig orderID extends Integer{}one sig CustomerOrderAssociation extends Association{}{src = Customerdst = Ordersrc_multiplicity = ONEdst_multiplicity = MANY}one sig ShippingCart extends Class{}{attrSet = shippingCartIDid=shippingCartIDisAbstract = Nono parent}one sig shippingCartID extends Integer{}one sig CustomerShippingCartAssociation extends Association{}{src = Customerdst = ShippingCartsrc_multiplicity = ONEdst_multiplicity = MANY}one sig Item extends Class{}{attrSet = ItemID+quantityid=ItemIDisAbstract = Nono parent}one sig ItemID extends Integer{}one sig quantity extends Integer{}one sig CartItem extends Class{}{attrSet = cartItemIDone parentid=ItemIDisAbstract = Noparent in Item}one sig cartItemID extends Integer{}one sig ShippingCartItemAssociation extends Association{}{src = ShippingCartdst = Itemsrc_multiplicity = ONEdst_multiplicity = MANY}one sig OrderItem extends Class{}{attrSet = orderItemID+statusone parentid=ItemIDisAbstract = Noparent in Item}one sig status extends Integer{}one sig orderItemID extends Integer{}one sig OrderItemAssociation extends Association{}{src = Orderdst = Itemsrc_multiplicity = ONEdst_multiplicity = MANY}one sig Category extends Class{}{attrSet = categoryID+categoryNameid=categoryIDisAbstract = Nono parent}one sig categoryID extends Integer{}one sig categoryName extends string{}one sig Product extends Class{}{attrSet = productID+productName+description+priceid=productIDisAbstract = Nono parent}one sig productID extends Integer{}one sig productName extends string{}one sig description extends string{}one sig price extends Real{}one sig ProductCategoryAssociation extends Association{}{src = Productdst = Categorysrc_multiplicity = MANYdst_multiplicity = MANY}one sig Catalog extends Class{}{attrSet = CatalogIDid=CatalogIDisAbstract = Nono parent}one sig CatalogID extends Integer{}one sig ProductCatalogAssociation extends Association{}{src = Productdst = Catalogsrc_multiplicity = ONEdst_multiplicity = MANY}one sig ProductItemAssociation extends Association{}{src = Productdst = Itemsrc_multiplicity = MANYdst_multiplicity = MANY}one sig PhysicalProduct extends Class{}{attrSet = weight+availabilityone parentid=productIDisAbstract = Noparent in Product}one sig weight extends Real{}one sig availability extends Bool{}one sig ElectronicProduct extends Class{}{attrSet = sizeone parentid=productIDisAbstract = Noparent in Product}one sig size extends string{}one sig Service extends Class{}{attrSet = scheduleone parentid=productIDisAbstract = Noparent in Product}one sig schedule extends string{}one sig Asset extends Class{}{attrSet = assetID+assetName+fileURIid = assetIDisAbstract = Nono parent}one sig assetID extends Integer{}one sig assetName extends string{}one sig fileURI extends string{}one sig ProductAssetAssociation extends Association{}{src = Productdst = Assetsrc_multiplicity = MANYdst_multiplicity = MANY}one sig Media extends Class{}{attrSet = mediaTypeone parentid = assetIDisAbstract = Noparent in Asset}one sig mediaType extends Integer{}one sig Documents extends Class{}{attrSet = excerptone parentid = assetIDisAbstract = Noparent in Asset}one sig excerpt extends string{}pred show{}run show for 48,Table Name: CustomerTable Name: OrderMapping Strategy for Customer : Union Sub ClassMapping Strategy for CartItem : Union Sub ClassMapping Strategy for OrderItem : Union Sub ClassMapping Strategy for Category : Union Sub ClassMapping Strategy for Product : Union Sub ClassMapping Strategy for PhysicalProduct : Union Sub ClassMapping Strategy for Asset : Union Sub ClassMapping Strategy for ElectronicProduct : Joined Sub ClassMapping Strategy for Service : Joined Sub ClassMapping Strategy for Service : Joined Sub ClassMapping Strategy for Documents : Joined Sub ClassAssociation Strategy for ShippingCartItemAssociation : ForeignKeyEmbeddingStrategyAssociation Strategy for CustomerOrderAssociation : OwnAssociationTableStrategyAssociation Strategy for CustomerShippingCartAssociation : OwnAssociationTableStrategyAssociation Strategy for OrderItemAssociation : OwnAssociationTableStrategyAssociation Strategy for ProductCategoryAssociation : OwnAssociationTableStrategyAssociation Strategy for ProductCatalogAssociation : OwnAssociationTableStrategyAssociation Strategy for ProductItemAssociation : OwnAssociationTableStrategyAssociation Strategy for ProductAssetAssociation : OwnAssociationTableStrategy,CREATE TABLE `Order` (`orderID` int NOT NULL,`customerID` int,KEY `FK_Order_customerID_idx` (`customerID`),PRIMARY KEY (`orderID`));CREATE TABLE `Category` (`categoryName` varchar(64),`categoryID` int NOT NULL,PRIMARY KEY (`categoryID`));CREATE TABLE `Customer` (`customerID` int NOT NULL,PRIMARY KEY (`customerID`));CREATE TABLE `Product` (`description` varchar(64),`productName` varchar(64),`price` decimal(20,5),`productID` int NOT NULL,PRIMARY KEY (`productID`));CREATE TABLE `Service` (`schedule` varchar(64),`productID` int NOT NULL,KEY `FK_Service_productID_idx` (`productID`),PRIMARY KEY (`productID`));CREATE TABLE `CartItem` (`cartItemID` int,`quantity` int,`ItemID` int NOT NULL,PRIMARY KEY (`ItemID`));CREATE TABLE `ShippingCart` (`shippingCartID` int NOT NULL,PRIMARY KEY (`shippingCartID`));CREATE TABLE `Catalog` (`CatalogID` int NOT NULL,PRIMARY KEY (`CatalogID`));CREATE TABLE `ProductItemAssociation` (`productID` int NOT NULL,`ItemID` int NOT NULL,KEY `FK_ProductItemAssociation_productID_idx` (`productID`),KEY `FK_ProductItemAssociation_ItemID_idx` (`ItemID`),PRIMARY KEY (`productID`,`ItemID`));CREATE TABLE `Item` (`quantity` int,`ItemID` int NOT NULL,`shippingCartID` int,KEY `FK_Item_shippingCartID_idx` (`shippingCartID`),PRIMARY KEY (`ItemID`));CREATE TABLE `ElectronicProduct` (`size` varchar(64),`description` varchar(64),`productName` varchar(64),`price` decimal(20,5),`productID` int NOT NULL,PRIMARY KEY (`productID`));CREATE TABLE `PhysicalProduct` (`availability` boolean,`weight` decimal(20,5),`productID` int NOT NULL,KEY `FK_PhysicalProduct_productID_idx` (`productID`),PRIMARY KEY (`productID`));CREATE TABLE `CustomerShippingCartAssociation` (`shippingCartID` int NOT NULL,`customerID` int NOT NULL,KEY `FK_CustomerShippingCartAssociation_shippingCartID_idx` (`shippingCartID`),KEY `FK_CustomerShippingCartAssociation_customerID_idx` (`customerID`),PRIMARY KEY (`shippingCartID`,`customerID`));CREATE TABLE `ProductAssetAssociation` (`assetID` int NOT NULL,`productID` int NOT NULL,KEY `FK_ProductAssetAssociation_assetID_idx` (`assetID`),KEY `FK_ProductAssetAssociation_productID_idx` (`productID`),PRIMARY KEY (`assetID`,`productID`));CREATE TABLE `OrderItem` (`orderItemID` int,`status` int,`ItemID` int NOT NULL,KEY `FK_OrderItem_ItemID_idx` (`ItemID`),PRIMARY KEY (`ItemID`));CREATE TABLE `Asset` (`DType` varchar(64),`excerpt` varchar(64),`fileURI` varchar(64),`assetName` varchar(64),`mediaType` int,`assetID` int NOT NULL,PRIMARY KEY (`assetID`));CREATE TABLE `ProductCatalogAssociation` (`CatalogID` int NOT NULL,`productID` int NOT NULL,KEY `FK_ProductCatalogAssociation_CatalogID_idx` (`CatalogID`),KEY `FK_ProductCatalogAssociation_productID_idx` (`productID`),PRIMARY KEY (`CatalogID`,`productID`));CREATE TABLE `OrderItemAssociation` (`ItemID` int NOT NULL,`orderID` int NOT NULL,KEY `FK_OrderItemAssociation_ItemID_idx` (`ItemID`),KEY `FK_OrderItemAssociation_orderID_idx` (`orderID`),PRIMARY KEY (`ItemID`,`orderID`));CREATE TABLE `ProductCategoryAssociation` (`productID` int NOT NULL,`categoryID` int NOT NULL,KEY `FK_ProductCategoryAssociation_productID_idx` (`productID`),KEY `FK_ProductCategoryAssociation_categoryID_idx` (`categoryID`),PRIMARY KEY (`productID`,`categoryID`));"
predicted_text = generate_predictions(input_text, model, tokenizer)
print("Predicted Output:", predicted_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted Output: the object-relational strategy represented by this sequence is the Pareto-optimal


In [ ]:
# Test the model with new input data
import pandas as pd

# Load CSV file
df = pd.read_excel("Bank_authentic_testset.xlsx")  # Update with your file path

# Assuming 'OM_Regular' is the column name
texts = df['OM_Regular']

# Iterate through each text in the column
for test_text in texts:

    predicted_text = generate_predictions(test_text, model, tokenizer)
    print("")
    print(f"status: {predicted_text}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

status: the object-relational strategy represented by this sequence is the Pareto-optimal

In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, classification_report

In [ ]:
dc = pd.read_excel('Bank_authentic_testset.xlsx')

In [ ]:
X_test2 = dc['OM_Regular'].values
y_test2 = dc['OM_Prediction'].values

In [ ]:
print(X_test2.shape)
print(y_test2.shape)

print("X data type: ", X_test2.dtype)
print("y data type: ", y_test2.dtype)

(32,)
(32,)
X data type:  object
y data type:  int64


In [ ]:
print(y_test2)

[1 1 1 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0]


In [ ]:
dd = pd.read_excel('Bank_authentic_testset_pred.xlsx')

In [ ]:
X_test_pred2 = dd['OM_Regular'].values
y_test_pred2 = dd['OM_Prediction'].values

In [ ]:
print (y_test_pred2 )

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
precision = precision_score(y_test2, y_test_pred2)
print("Testing: Precision = %f" % precision)


recall = recall_score(y_test2, y_test_pred2)
print("Testing: Recall = %f" % recall)


f1 = f1_score(y_test2, y_test_pred2)
print("Testing: F1 Score = %f" % f1)

print("\nConfusion Matrix (Test Data):\n", confusion_matrix(y_test2, y_test_pred2))

Testing: Precision = 0.593750
Testing: Recall = 1.000000
Testing: F1 Score = 0.745098

Confusion Matrix (Test Data):
 [[ 0 13]
 [ 0 19]]


In [ ]:
print(classification_report(y_test2,y_test_pred2))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.59      1.00      0.75        19

    accuracy                           0.59        32
   macro avg       0.30      0.50      0.37        32
weighted avg       0.35      0.59      0.44        32



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
